# CHSH game
CHSH game is not a troditional notion,it falls within a class of games known as nonlocal games.

## Nonlocal games

### Description
Nonlocal game describes a scenario where plays may communicate and strategize before the game starts,but once it begins, they are strictly isolated from each other.
Imagining that a referee seperate Alice and Bob into two rooms. The referee send an input 'x' to Alice and 'y' to Bob. They must then output a output 'a' and 'b' respectively.


  No communication between Alice and Bob
                 
       +-------+     |     +-------+
       |       |     |     |       |
       | Alice |     |     |  Bob  |
       |       |     |     |       |
       +-------+     |     +-------+
        ^     |      |      ^     |
      x |     | a    |    y |     | b
        |     v      |      |     v
       +-------------+-------------+
       |                           |
       |          Referee          |
       |                           |
       +---------------------------+


According the rules,they win the game if and only if the inputs and outputs satisfy  '$a \oplus b = x \land y$'. According to the equation, we can get the table below:

| $(x, y)$ | win | lose |
| :---: | :---: | :---: |
| $(0, 0)$ | $a = b$ | $a \neq b$ |
| $(0, 1)$ | $a = b$ | $a \neq b$ |
| $(1, 0)$ | $a = b$ | $a \neq b$ |
| $(1, 1)$ | $a \neq b$ | $a = b$ |

Let's analyse this table. When the question is (x=0,y=0),Alice and Bob win only if their outputs is equal,i.e.,a(0)=b(0)(Here,we define a(x) as Alice's output when she receives input 'x'). And we also get :
* (x=0,y=1) --->  a(0) = b(1)
* (x=1,y=0) --->  a(1) = b(0)

If they use a strategy where they always give the same output (e.g., always 0), they win in the first three cases. But they fail when (x=1,y=1).So we can calculate their maximum winning probability is 75%.
Can probabilistic strategies or statistical methods (like MLE) break this limit?
No. Since any randomized strategy is simply a weighted average of deterministic ones, the win rate cannot exceed the theoretical maximum of 75%.

### How about a quantum strategy?

             +------ |ψ> ------+
             |                 |
             v                 v
       +-------+     |     +-------+
       |       |     |     |       |
       | Alice |     |     |  Bob  |
       |       |     |     |       |
       +-------+     |     +-------+
        ^     |      |      |     ^
      x |     | a    |    b |     | y
        |     v      |      v     |
       +---------------------------+
       |                           |
       |          Referee          |
       |                           |
       +---------------------------+

Let's define a quantum state vector $|\psi_{\theta}\rangle$ as below:($\theta$ is a real number)
$$|\psi_{\theta}\rangle = \cos(\theta)|0\rangle + \sin(\theta)|1\rangle$$
The inner product between two such states is given by :
$$\langle \psi_{\alpha} | \psi_{\beta} \rangle = \cos(\alpha)\cos(\beta) + \sin(\alpha)\sin(\beta) = \cos(\alpha - \beta).$$

Then,we define a unitary matrix $U_{\theta}$ for each angle $\theta$ as follows:

$$U_{\theta} = |0\rangle\langle \psi_{\theta}| + |1\rangle\langle \psi_{\theta+\pi/2}|$$
This operate maps $|\psi_{\theta}\rangle$ to $|0\rangle$ and $|\psi_{\theta+\pi/2}\rangle$ to $|1\rangle$. And it is straightforward to verify it's a unitary matrix.
We can also represent  $U_{\theta}$ in metrix form as follows(Note:A matrix which satisfies orthogonality and has a determinant of 1 is a rotate matrix.):
$$U_{\theta} = \begin{pmatrix} \cos(\theta) & \sin(\theta) \\ \cos(\theta+\pi/2) & \sin(\theta+\pi/2) \end{pmatrix} = \begin{pmatrix} \cos(\theta) & \sin(\theta) \\ -\sin(\theta) & \cos(\theta) \end{pmatrix}.$$

### How to prepare?
Alice holds qubit A, and Bob holds B. We prepare A and B in the entangled state,and are in $|\phi^{+}\rangle$.
* If Alice receives question x=0, she applies $U_0$ to qubit A(x=1,applies $U_{\pi/4}$).
* If Bob receives question x=0, he applies $U_{\pi/8}$ to qubit B(y=1,applies $U_{-\pi/8}$).

Let's see what will happen in case 'x=0,y=0':
The system is described by the formula 
$$(U_0 \otimes U_{\pi/8}) |\phi^{+}\rangle = |00\rangle\langle \psi_{0} \otimes \psi_{\pi/8}|\phi^{+}\rangle + |01\rangle\langle \psi_{0} \otimes \psi_{5\pi/8}|\phi^{+}\rangle + |10\rangle\langle \psi_{\pi/2} \otimes \psi_{\pi/8}|\phi^{+}\rangle + |11\rangle\langle \psi_{\pi/2} \otimes \psi_{5\pi/8}|\phi^{+}\rangle 
$$
$$
= \frac{\cos(-\pi/8)|00\rangle + \cos(-5\pi/8)|01\rangle + \cos(3\pi/8)|10\rangle + \cos(-\pi/8)|11\rangle}{\sqrt{2}}.$$
We have known that in this case Alice and Bob will win if the outputs are the same.
$$\Pr(a = b) = \frac{2 + \sqrt{2}}{4}
$$
$$
\Pr(a \neq b) = \frac{2 - \sqrt{2}}{4}$$
In the other three cases,the calculation yields the same winning probability.Finally,the average probability of win is $\frac{2 + \sqrt{2}}{4} \approx 0.85$(Note:there is a slightly difference between Case 4 and the others, but the final result is the same. ).

In [ ]:
from qiskit import QuantumCircuit
import random
from math import pi
from qiskit_aer import AerSimulator

In [ ]:
def chsh_game(strategy):
    # This function runs the CHSH game, using the strategy (a function
    # from two bits to two bits), returning 1 for a win and 0 for a loss.
 
    # Choose x and y randomly
    x, y = random.randint(0, 1), random.randint(0, 1)
 
    # Use the strategy to determine a and b
    a, b = strategy(x, y)
 
    # Decide if the strategy wins or loses
    if (a != b) == (x & y):
        return 1  # Win
    return 0  # Lose

In [ ]:
def chsh_circuit(x, y):

    qc = QuantumCircuit(2, 2)
 
    # Prepare an e-bit
    qc.h(0)
    qc.cx(0, 1)
    qc.barrier()
 
    # Alice's actions
    if x == 0:
        qc.ry(0, 0)
    else:
        qc.ry(-pi / 2, 0)
    qc.measure(0, 0)
 
    # Bob's actions
    if y == 0:
        qc.ry(-pi / 4, 1)
    else:
        qc.ry(pi / 4, 1)
    qc.measure(1, 1)
 
    return qc

# Draw the four possible circuits
 
print("(x,y) = (0,0)")
display(chsh_circuit(0, 0).draw(output="mpl"))
 
print("(x,y) = (0,1)")
display(chsh_circuit(0, 1).draw(output="mpl"))
 
print("(x,y) = (1,0)")
display(chsh_circuit(1, 0).draw(output="mpl"))
 
print("(x,y) = (1,1)")
display(chsh_circuit(1, 1).draw(output="mpl"))

### quantum stratege vs classical strategy


In [ ]:
def quantum_strategy(x, y):
    # This function runs the appropriate quantum circuit defined above
    # one time and returns the measurement results
 
    # Setting `shots=1` to run the circuit once
    result = AerSimulator().run(chsh_circuit(x, y), shots=1).result()
    statistics = result.get_counts()
 
    # Determine the output bits and return them
    bits = list(statistics.keys())[0]
    a, b = bits[0], bits[1]
    return a, b

def classical_strategy(x, y):
    # This function implements just one example of an optimal classical
    # strategy for the CHSH game. Other classical strategies can be
    # implemented by changing the bit values assigned to a and b.
 
    # Alice's answer
    if x == 0:
        a_c = 0
    elif x == 1:
        a_c = 1
 
    # Bob's answer
    if y == 0:
        b_c = 1
    elif y == 1:
        b_c = 0
 
    return a_c, b_c

NUM_GAMES = 1000
TOTAL_SCORE = 0
for _ in range(NUM_GAMES):
    TOTAL_SCORE += chsh_game(quantum_strategy)


print("Fraction of games won:", TOTAL_SCORE / NUM_GAMES)
TOTAL_SCORE = 0
for _ in range(NUM_GAMES):
    TOTAL_SCORE += chsh_game(classical_strategy)
 
print("Fraction of games won:", TOTAL_SCORE / NUM_GAMES)

# !!! Note:
You may be confused about why the Qiskit code chooses the angle set $\{0, \pi/2, -\pi/4, \pi/4\}$, as it differs from the theoretical derivation. This is because in classical geometry, the orthogonal angle is $\pi/2$, but on the Bloch Sphere, $|0\rangle$ and $|1\rangle$ differ by $\pi$.